# FINMA Python Lab 10A: Vanilla Monte Carlo to Reproduce Black-Scholes

## Overview

In this lab, you'll learn how to price options using the **simplest form of Monte Carlo simulation** and verify your results against the famous **Black-Scholes formula**.

### What You'll Learn:
1. The basic concept of Monte Carlo simulation
2. How to simulate stock prices
3. How to price vanilla call and put options
4. How to reproduce Black-Scholes results
5. How many simulations you need for accuracy

**Prerequisites:** Basic NumPy, understanding of options

**Important:** Complete your work and have it manually checked by your instructor.

---


## Programming Exercises

### Exercise 1: Build Monte Carlo from Scratch

Write your own Monte Carlo pricer **without looking at the function above**.

Follow these steps:
1. Generate random numbers Z
2. Calculate final stock prices ST
3. Calculate payoffs
4. Average and discount

**Test:** Price a call with S0=100, K=105, T=0.5, r=0.05, sigma=0.25

**Expected:** Should be close to $6.04

#### Step 1: Set up parameters and import libraries

First, let's define all the parameters we need.

In [ ]:
import numpy as np

# Define parameters for our test
S0 = 100      # Initial stock price
K = 105       # Strike price
T = 0.5       # Time to maturity (years)
r = 0.05      # Risk-free rate
sigma = 0.25  # Volatility
n_sims = 10000  # Number of Monte Carlo simulations

print(f"Parameters set!")
print(f"S0={S0}, K={K}, T={T}, r={r}, sigma={sigma}")
print(f"Number of simulations: {n_sims}")

#### Step 2: Generate random numbers Z

In Monte Carlo, we simulate randomness using standard normal random variables.

**Hint:** Use `np.random.randn()` to generate `n_sims` random numbers from a standard normal distribution (mean=0, std=1).

**Key Formula:** Under the Black-Scholes model:
$$S_T = S_0 \times e^{(r - 0.5\sigma^2)T + \sigma\sqrt{T}Z}$$

where Z ~ N(0,1)

In [ ]:
# TODO: Generate n_sims random numbers from standard normal distribution
# Z = ...

# Example structure:
# Z = np.random.randn(???)

print(f"Generated {len(Z)} random numbers")
print(f"First 5 random numbers: {Z[:5]}")

#### Step 3: Calculate final stock prices ST

Now use the random numbers Z to calculate what the stock price will be at time T for each simulation.

**Formula:** 
$$S_T = S_0 \times e^{(r - 0.5\sigma^2)T + \sigma\sqrt{T}Z}$$

**Breakdown:**
- `drift` = $(r - 0.5\sigma^2)T$
- `diffusion` = $\sigma\sqrt{T}Z$
- $S_T = S_0 \times e^{drift + diffusion}$

**Hint:** Use `np.exp()` for the exponential function.

In [ ]:
# TODO: Calculate the final stock prices
# Step by step:
# 1. Calculate drift = (r - 0.5 * sigma**2) * T
# 2. Calculate diffusion = sigma * np.sqrt(T) * Z  
# 3. Calculate ST = S0 * np.exp(drift + diffusion)

# drift = ...
# diffusion = ...
# ST = ...

print(f"Calculated {len(ST)} final stock prices")
print(f"First 5 stock prices: {ST[:5]}")
print(f"Min: ${ST.min():.2f}, Max: ${ST.max():.2f}, Mean: ${ST.mean():.2f}")

#### Step 4: Calculate payoffs

For a **call option**, the payoff at maturity is:
$$\text{Payoff} = \max(S_T - K, 0)$$

This means:
- If $S_T > K$: the option is in-the-money, payoff = $S_T - K$
- If $S_T \leq K$: the option is worthless, payoff = $0$

**Hint:** Use `np.maximum()` which takes element-wise maximum of two arrays.

In [ ]:
# TODO: Calculate call option payoffs
# payoffs = np.maximum(ST - K, 0)

# payoffs = ...

print(f"Calculated {len(payoffs)} payoffs")
print(f"First 10 payoffs: {payoffs[:10]}")
print(f"Number of ITM options: {(payoffs > 0).sum()} out of {n_sims}")
print(f"Average payoff: ${payoffs.mean():.4f}")

#### Step 5: Average and discount

The next step is to:
1. Calculate the **average** of all payoffs
2. **Discount** it back to present value

**Formula:**
$$\text{Option Price} = e^{-rT} \times \text{Average Payoff}$$

**Why discount?** Because the payoff happens in the future (at time T), we need to calculate its present value today.

**Hint:** 
- Use `.mean()` to average the payoffs
- Use `np.exp(-r * T)` for the discount factor

In [ ]:
# TODO: Calculate the option price
# 1. Calculate the average payoff
# 2. Calculate the discount factor
# 3. Multiply them together

# discount_factor = np.exp(-r * T)
# option_price = discount_factor * payoffs.mean()

# discount_factor = ...
# option_price = ...

print(f"="*50)
print(f"MONTE CARLO OPTION PRICING RESULT")
print(f"="*50)
print(f"Discount factor: {discount_factor:.6f}")
print(f"Average payoff: ${payoffs.mean():.4f}")
print(f"Option price: ${option_price:.4f}")
print(f"="*50)
print(f"Expected: ~$6.04")
print(f"Your result: ${option_price:.4f}")
print(f"Difference: ${abs(option_price - 6.04):.4f}")

#### Step 6: Bring it all together

The final step is to put everything above into one, reusable function that's flexible with different params

In [ ]:
# Exercise 1: Write your Monte Carlo pricer here into ONE FUNCTION now



---

### Comparing with Black-Scholes

The **Black-Scholes formula** is the famous analytical solution for European option prices.

Let's see if our Monte Carlo simulation matches it!

In [ ]:
from scipy.stats import norm

def black_scholes_call(S0, K, T, r, sigma):
    """
    Black-Scholes formula for European call option.
    
    Don't worry about understanding this formula - just use it!
    """
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Calculate Black-Scholes price
bs_price = black_scholes_call(S0, K, T, r, sigma)

# Our Monte Carlo price
mc_price = option_price

print(f"OPTION PRICING COMPARISON:")
print(f"="*50)
print(f"Black-Scholes (exact):  ${bs_price:.4f}")
print(f"Monte Carlo (10,000):   ${mc_price:.4f}")
print(f"="*50)
print(f"Difference:             ${abs(bs_price - mc_price):.4f}")
print(f"Error:                  {abs(bs_price - mc_price) / bs_price * 100:.2f}%")

if abs(bs_price - mc_price) / bs_price < 0.02:  # Within 2%
    print(f"\n✓ Excellent! Monte Carlo matches Black-Scholes!")
else:
    print(f"\n⚠ Try increasing number of simulations for better accuracy")

**Success!** Our simple Monte Carlo simulation reproduces the Black-Scholes price! 🎉

### Exercise 2: Test Different Strike Prices

Price call options with these strike prices: K = [80, 90, 100, 110, 120]

For each strike:
1. Calculate Monte Carlo price (10,000 simulations)
2. Calculate Black-Scholes price
3. Calculate the error
4. Create a table showing results

**Parameters:** S0=100, T=1, r=0.05, sigma=0.2

**Question:** Which strike has the largest absolute error? Why?

In [ ]:
# Exercise 2: Test different strikes



### Exercise 3: Convergence Study

Run the SAME Monte Carlo pricing **50 times** with 5,000 simulations each.

1. Store all 50 prices in a list
2. Calculate the mean and standard deviation of these 50 prices
3. Create a histogram of the 50 prices

**Parameters:** S0=100, K=100, T=1, r=0.05, sigma=0.2

**Question:** How much do the Monte Carlo prices vary?

In [ ]:
# Exercise 3: Convergence study



### Exercise 4: Put-Call Parity

**Put-Call Parity** is a fundamental relationship:

$$C - P = S_0 - K \times e^{-rT}$$

Where C = call price, P = put price

Verify this relationship:
1. Price a call using Monte Carlo
2. Price a put using Monte Carlo
3. Calculate the left side: C - P
4. Calculate the right side: S0 - K*exp(-rT)
5. Compare them

**Parameters:** S0=100, K=100, T=1, r=0.05, sigma=0.2, n_sims=50000

**Question:** Do they match? How close?

In [ ]:
# Exercise 4: Put-Call Parity



### Exercise 5: Volatility Impact

Test how volatility affects option prices.

Price call options with volatilities: sigma = [0.1, 0.2, 0.3, 0.4, 0.5]

1. For each sigma, price the option with Monte Carlo
2. Also calculate Black-Scholes price
3. Create a plot showing both MC and BS prices vs volatility
4. Add a second plot showing the error vs volatility

**Parameters:** S0=100, K=100, T=1, r=0.05, n_sims=20000

**Question:** Does Monte Carlo accuracy depend on volatility level?

In [ ]:
# Exercise 5: Volatility impact



### Exercise 6: Time to Maturity

Test how time to maturity affects option prices.

Price call options with maturities: T = [0.25, 0.5, 0.75, 1.0, 1.5, 2.0]

1. For each T, price with both MC and BS
2. Plot option price vs time to maturity
3. Calculate and plot the error vs time

**Parameters:** S0=100, K=100, r=0.05, sigma=0.2, n_sims=20000

**Question:** Does Monte Carlo work better for short-dated or long-dated options?

In [ ]:
# Exercise 6: Time to maturity



### Exercise 7: In-the-Money Probability

For a call option, calculate:
1. The probability it expires in-the-money (ITM)
2. The expected payoff given it expires ITM
3. The expected payoff given it expires out-of-the-money (OTM)

Use Monte Carlo with different strikes: K = [90, 100, 110]

Create a table with columns:
- Strike
- ITM Probability
- Expected Payoff | ITM
- Option Price

**Parameters:** S0=100, T=1, r=0.05, sigma=0.2, n_sims=50000

In [ ]:
# Exercise 7: ITM probability analysis

